# Single pixel sampler 

This notebook contains a Marcov Chain Monte Carlo (MCMC) sampler and a Gibbs sampler for doing component separation for a singel pixel. By sampling over only one pixel we can easily do tens of thousands of iterations quickly. Thus we aim to give an intuitive understanding of how Commander3 works without having to go through all the Commander code in details. To see the full details on Commander3, please see the [main BeyondPlanck paper](https://arxiv.org/pdf/2011.05609.pdf) and the acompanying papers, as well as the [Commander3 code itself](https://github.com/Cosmoglobe/Commander).

An overview of the three sessions are as follows:
1. The sky model
2. Markov chain Monte Carlo (MCMC) sampling and priors
3. Gibbs sampling

# 1: Sky model
First we will introduce the data model for the polarized sky containing CMB (Cosmic Microwave Backround), synchrotron and thermal dust. 

### CMB
For the CMB radiation we have 
$$s_{\mathrm{CMB}} = a_{\mathrm{CMB}} \frac{x^2e^{x}}{\left(e^{x}-1\right)^2}$$
where $a_{\mathrm{CMB}}$ is the strength (amplitude) of the signal and the rest is a converstion factor between Rayleigh Jeans units and CMB units where  $x=h\nu/kT_0$.

### Synchrotron
The synchrotron radiation dominates the lower frequency bands. We use a power law model given by
$$s_{\mathrm{s}} = a_{\mathrm{s}} \left(\frac{\nu}{\nu_{0,\mathrm{s}}} \right)^{\beta_{\mathrm{s}}+ C \log\left(\frac{\nu}{\nu_{0,\mathrm{s}}}\right)}$$
where $a_{\mathrm{s}}$ is the strength of the signal at the reference frequency $\nu_{0,\mathrm{s}}$, $\beta_{\mathrm{s}}$ is the spectral index and $C$ is the curvature. 

### Thermal dust
Thermal dust dominates the higher frequencies. Dust is modelled with a modified blackbody curve, 
$$s_{\mathrm{d}} = a_{\mathrm{d}}\,\left(\frac{\nu}{\nu_{0,\mathrm{d}}}\right)^{\beta_{\mathrm{d}}+1}
\frac{e^{h\nu_{\mathrm{0,\mathrm{d}}}/kT_{\mathrm{d}}}-1}{e^{h\nu/kT_{\mathrm{d}}}-1}$$
where $a_{\mathrm{d}}$ is the strength of the signal at the reference frequency $\nu_{0,\mathrm{d}}$, $\beta_{\mathrm{d}}$ is the spectral index and $T_{\mathrm{d}}$ is the dust temperature. 

## Full sky model 
The sky model $s_{\mathrm{RJ}}$ that we will use is thus given by this model 
$$s_{\mathrm{RJ}} =s_{\mathrm{CMB}} + s_{\mathrm{s}}+ s_{\mathrm{d}}$$ containing a total of seven parameters to be sampled. Three amplitudes ($a_{\mathrm{CMB}}$, $a_{\mathrm{s}}$ and $a_{\mathrm{d}}$), two spectral indexes ($\beta_{\mathrm{s}}$ and $\beta_{\mathrm{s}}$), synchrotron curvature ($C$) and thermal dust ($T_{\mathrm{d}}$). We therefore need at least 7 frequency bands to fit this model. 

Using these models we can write up functions that calculates the synchrotron signal, the thermal dust signal and the unit converter for a given frequency. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
random.seed(420)

# Constants
kb   = 1.38e-23      # Boltzmanns constant
Tcmb = 2.7255        # CMB temperature
h    = 6.626e-34     # Plancks constant

# Calculates synchrotron value for given input frequency and parameters
def synch_spec(A_synch, b_synch, synch_curv, synch_freq, synch_ref):
    return float(A_synch*((synch_freq/synch_ref)**(b_synch+synch_curv*np.log(synch_freq/synch_ref))))

# Calculates dust value for given input frequency and parameters
def dust_spec(A_dust, b_dust, T_dust, dust_freq, dust_ref):
    return float(A_dust*((dust_freq/dust_ref)**(b_dust + 1.)))* \
        (np.exp(h*dust_ref*1.0e9/(kb*T_dust))-1.)/ \
        (np.exp(h*dust_freq*1.0e9/(kb*T_dust))-1.)

# Unit conversion between Rayleigh–Jeans units and cmb units
def unit_conv(frequency):
    xx = h*frequency*1.0e9/(kb*Tcmb)
    return xx**2*np.exp(xx)/((np.exp(xx)-1.)**2)

## Simulated sky data
Now we can simulate our data using this sky model for a given set of frequencies. Below we set the model parameters, the set of frequencies, the level of noise for each frequency and which frequency we use as pivot frequency (reference frequency) for dust and syncrotron. 

We have added all the Planck frequencies although not all of them were sensitive to polarization. Feel free to add the frequencies and sensitivities from your favourite experiment.

In [ ]:
#Pixel size. Small number => large pixels => little noise pr pixel
nside = 32
#Frequency bands for Planck
freq = np.array([30., 44., 70., 100., 143., 217., 353., 545., 857.])
#Noise level pr pixel, pr frequency
sigma = np.array([1.2, 2.7, 3.2, 5.6, 2.1, 3.8, 1.3, 2.9, 1.7])*nside/512 
# Choose pivot frequencies
synch_ref = freq[0]   #Pivot/reference frequency synchrotron
dust_ref = freq[-3]   #Pivot frequency dust

#Model parameters for given pivot frequencies
Acmb   = .67     # CMB amplitude
As     = 12      # Synchrotron amplitude at the reference frequency
Betas  = -3.1    # Synchrotron beta (spectral index)
Csynch = -0.39   # Synchrotron curvature (set to zero if you do not want to include it)
Ad     = 8       # Dust amplitude at the reference frequency
Betad  = 1.6     # Dust beta (spectral index)
Tdust  = 18.5    # Dust temperature
names  = (["Acmb", "As", "Betas", "Csynch", "Ad", "Betad", "Tdust"])
param_in = np.array([Acmb, As, Betas, Csynch, Ad, Betad, Tdust])

# empty arrays for storing the data model for each frequency band
cmb   = np.empty(np.size(freq)) 
synch = np.empty(np.size(freq))
dust  = np.empty(np.size(freq))
data  = np.empty(np.size(freq))

print ("freq  | ","cmb   | ", "synch | ", "dust  | ", "data")
#Generate our data set for each frequenxy
for i in range(np.size(freq)):
    # Make CMB data set
    cmb[i] = Acmb*unit_conv(freq[i])
    # Synchrotron data
    synch[i] = synch_spec(As, Betas, Csynch, freq[i], synch_ref)
    # Dust data
    dust[i] = dust_spec(Ad, Betad, Tdust, freq[i], dust_ref)
    # Full data set
    data[i] = cmb[i] + synch[i] + dust[i]
    print (freq[i], " | ", format(cmb[i], '.2f'), " | ", format(synch[i], '.2f'), " | ", format(dust[i], '.2f'), " | ", format(data[i], '.2f'))
    
# Add instrumental noise to signal
data = data + np.random.randn(len(sigma))*sigma

In [ ]:
# Plot cmb, synchrotron, dust and total signal
def plot_func(cmb_values, synch_values, dust_values, data_values, linestyle="-", tag=""):
    plt.plot(freq.T, cmb_values, label = "CMB "+tag, linestyle=linestyle, color="C0",)
    plt.plot(freq.T, synch_values, label = "Synchrotron "+tag, linestyle=linestyle,color="C1",)
    plt.plot(freq.T, dust_values, label = "Thermal dust "+tag, linestyle=linestyle,color="C2",)
    plt.plot(freq.T, data_values, label = "Sum fg. "+tag, linestyle=linestyle,color="C3",)
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel("Frequency [GHz]")
    plt.ylabel("Amplitude [uK_RJ]")
    plt.ylim(1e-2,1e2)
    plt.legend(loc="best", frameon=False)

plot_func(cmb, synch, dust, data)
#Data points for task 1.2:
#plt.plot(freq, [12.2, 9.0, 5.4, 4.5, 4.7, 4.8, 8.2, 13.1, 19.3], 'kx')
plt.show()

## Tasks 1
1. Play around changing the parameters in the model, the pixel size and the reference frequency and get an understanding on what changing each parameter does to the full sky model. 
2. Comment in the data points for task 1.2. Change the parameters so that the total signal (red curve) in the plot above matches these data points marked in the plot. 

# 2: MCMC sampling

We will now use the above "data" array as our simulated sky for one sky pixel. This data includes cmb, foregrounds and noise. The next step is trying to fit our sky model to our data. Since we know what the input was, we already know what the true answer should be. Thus we can use this knowledge to judge wether our sampler is able to reproduce out input. 

First we will use Markov chain Monte Carlo (MCMC) sampling, please have a look look at for instance [this blog on MCMC sampling for dummies](https://twiecki.io/blog/2015/11/10/mcmc-sampling/) or other sources if you are unfamiliar with this method. 

For initializing the sampler, we set the number of samples we want and how long the burn-in period is. The burn in period os how many samples we leave out from the beginning of the sampling chain before the sampling has stabilized. We leave these first samples out when generating the probability distribution for the parameter values. We also set a starting point for our parameter values, this is our best guess for the value. In real life we do not know the true sky model but we have estimations from previous experiments. The step size for each parameter are also set. 

In [ ]:
it = 100000           # number of samples in a chain
burn = 1000          # period of burn in
nr_values = 7       # number of parameters to sample

samples    = np.empty([it, nr_values])  # Array for storing samples
teta_start = np.empty([nr_values])      # Array for current sample
steps      = np.empty([nr_values])      # Array for step sizes

# Set _starting_ value for each parameter, this is our "best guess" of the value
teta_start[0] = .5 #.67      # CMB amplitude
teta_start[1] = 8 #12.0     # Synchrotron amplitude
teta_start[2] = -3.0 #-3.1     # Synchrotron beta (spectral index)
teta_start[3] = 0 #-0.39    # Synchrotron curvature
teta_start[4] = 10 #8.0      # Dust amplitude
teta_start[5] = 1.7 #1.6      # Dust beta (spectral index)
teta_start[6] = 20 #18.5     # Dust temperature

# Set step size for each parameter. Set to zero to keep a parameter fixed to the starting value. 
steps[0] = 0.05    # CMB amplitude
steps[1] = 0.1     # Synchrotron amplitude
steps[2] = 0.04    # Synchrotron beta (spectral index)
steps[3] = 0.04    # Synchrotron curvature
steps[4] = 0.5     # Dust amplitude
steps[5] = 0.05     # Dust beta (spectral index)
steps[6] = 0.8    # Dust temperature

## Priors
In bayesian statistics we use our prior knowledge about our parameters. Here we only include flat priors, but more advanced priors can of course be implemented, and is implementet in Commander3. Examples are gaussian priors or Jeffreys prior. For our three amplitudes, we discard negative values to ensure positive amplitudes. For dust temperature and the betas we use our knowledge from prior experiments to set allowed intervals. These should not be too small intervals. 

In [ ]:
#Flat prior to exclude too extreme values and negative amplitudes
def flat_prior(tet):
    test = 0.
    if tet[0] < 0 or tet[1]<0 or tet[4]<0: # Ensure positive amplitudes
        test = -float("inf")
    if tet[2] < -4.2 or tet[2] > -2.0:   # synch Beta
        test = -float("inf")
    if tet[3] < -1.0 or tet[3] > 0.5:
        test = -float("inf")
    if tet[5] < 0.5 or tet[5] > 3.5:     # dust beta
        test = -float("inf")
    if tet[6] < 1.0 or tet[6] > 30.:    # dust temp
        test = -float("inf")
    return test

## Sampling

In [ ]:
step_matrix = np.diag(steps)
teta0 = teta_start.copy()
teta  = teta_start.copy()
n = 0.              # for calculating acceptance rate    

#calculates probability that teta is the correct values for our model, 
#given the simulated data for each frequency and our foreground model
#Normal distribution
def prob(tet, dat, sig, fr, frs, frd):
    P = 0.
    for i in range(np.size(dat)): # Loop over frequencies)
        uc = unit_conv(fr[i])   # unit convertion
        cm = float(tet[0]*uc)  # cmb
        s = synch_spec(tet[1], tet[2], tet[3], fr[i], frs) #synchotron
        d = dust_spec(tet[4], tet[5], tet[6], fr[i], frd) #dust
        t = float(cm + s + d)
        P = P-((dat[i]-t)/sig[i])**2
    return P/2.

#probability for starting values:
old_prob = prob(teta, data, sigma, freq, synch_ref, dust_ref) + flat_prior(teta) 

for i in range(it):
    teta = teta + np.dot(np.random.randn(nr_values),step_matrix) #suggest new values
    prop_prob = prob(teta, data, sigma, freq, synch_ref, dust_ref) + flat_prior(teta) #probability for new teta
    #Calculate difference in probability for old and new values.
    A = min(1., np.exp(prop_prob - old_prob))
    #Always accept more probable values, sometimes accept worse vcalues
    if A >= np.random.uniform(0,1):
        teta0 = teta.copy()
        old_prob = prop_prob
        n += 1
    else: 
        teta = teta0.copy()
    samples[i, :] = teta

#New array without the first values, before burn in
samples_burn = samples[burn:, :]

print ("Accenptance rate: ", n/it)

print ("Parameter", " | ", "std ", " | ", "Average", " | ", "True value")
for i in range(nr_values):
    print ('{:>9}'.format(names[i]), " | ", format(np.std(samples_burn[:,i]), '.2f'), " | ", format(np.average(samples_burn[:,i]), '.2f'), " | ", param_in[i])

## Plotting sampling timeline
In an ideal world, the sampler will converge to a value and explore the parameter space around that value.
- If the timelines have long plataues (several samples with the same value): the step size is too long for one of the parameters.
- If the timeline looks very random with no values seeming to be prefered: the step size is too small. 

The acceptance rate printed above is also an idication on our choices for step sizes. This number should idealy be around 50% for a 1 parameter model, but less for a 7 parameter model.

In [ ]:
for i in range(np.size(names)):
    plt.plot(samples[:,i])
    plt.title(names[i])
    plt.vlines(burn, samples[:,i].min(), samples[:,i].max(), label= "burnin", colors="r")
    plt.hlines(param_in[i],0, it, label = "Input value", colors = "c")
    plt.legend(loc="upper right")
    plt.show()

## Plotting histogram for all samples 

In [ ]:
for i in range(np.size(names)):
    y, x, _ = plt.hist(samples_burn[:,i], bins=min(50, int(np.sqrt(it))))
    plt.vlines(param_in[i], 0, y.max(), label = "Input value", colors = "c")
    plt.vlines(np.average(samples_burn[:,i]), 0, y.max(), label="Average", colors="r")
    plt.title(names[i])
    plt.legend(loc="upper right")
    plt.show()
    print (" - - - - - - - - - - - - - ")
    plt.clf()

## Plotting model based on samples

In [ ]:
cmb_bf   = np.empty(np.size(freq)) 
synch_bf = np.empty(np.size(freq))
dust_bf  = np.empty(np.size(freq))
for i in range(np.size(freq)):
    # Make CMB data set
    cmb_bf[i] = np.average(samples_burn[:,0])*unit_conv(freq[i])
    # Synchrotron data
    synch_bf[i] = synch_spec(np.average(samples_burn[:,1]), np.average(samples_burn[:,2]), np.average(samples_burn[:,3]), freq[i], synch_ref)
    # Dust data
    dust_bf[i] = dust_spec(np.average(samples_burn[:,4]), np.average(samples_burn[:,5]), np.average(samples_burn[:,6]), freq[i], dust_ref)
plot_func(cmb_bf, synch_bf, dust_bf, cmb_bf+synch_bf+dust_bf, linestyle=":", tag="MCMC")
plot_func(cmb, synch, dust, data)
plt.show()

## Tasks 2:
1. Play around with the step sizes in "steps" to see how this changes the timeline plots.
2. Based on the timeline plots: evaluate how many samples should be left out in the beginning (burn)
3. Change the values for for instance  the nside, number of samples, starting value for sampler, flat prior limits etc. to see hoe this changes the final result 

# 3: Gibbs sampler

In [ ]:
import sys
import os
import matplotlib.pylab as plt
import numpy as np
import scipy.stats as stats
from scipy.interpolate import interp1d
import corner
from tqdm import tqdm
class sky_model:
    #np.random.seed(1234)
    """
    This model is a sky signal using two sky components. It has four free
    parameters. The CMB component has an amplitude A_cmb.
    And the thermal dust component had an amplitude A_d relative to the
    reference frequency nu_0, a spectral index beta which determines the 
    slope of the distribution, and the temperature T spectral index, 
    which determines the peak of the spectrum.
    """
    def __init__(self,nus):
        # These parameters represent the true simulated sky
        self.nus   = nus                 # Frequencies
        self.A_d   = 100.                # Amplitude in muK
        self.A_cmb = 67.                 # Amplitude in muK
        self.beta  = 1.6                 # Dust spectral index
        self.T     = 19.                 # Dust spectral index in muK
        self.nu_0  = 857.                # GHz er en fiksert referanse-frekvens
        self.h     = 6.62607e-34         # Planck's comstant
        self.k_b   = 1.38065e-23         # Boltzmanns constant
        self.Tcmb  = 2.7255              # K CMB Temperature
        self.K     = self.h*1e9/self.k_b # Constant

    def s_nu(self, nu):
        # Total signal as a function of frequency
        return self.s_d(nu)+self.s_cmb(nu)

    def s_d(self,nu): 
        # Thermal dust modified blackbody
        return self.A_d*(nu/self.nu_0)**(self.beta+1)*(np.exp(self.K*self.nu_0/self.T) - 1.)/(np.exp(self.K*nu/self.T) - 1.)

    def s_cmb(self,nu): 
        # CMB blackbody
        return self.A_cmb/self.g(nu)

    def g(self,nu): 
        # Conversion factor between thermodynamic and brightness temperature
        x = self.K*nu/self.Tcmb
        return (np.exp(x)-1)**2 / (x**2 * np.exp(x))

    def d_nu(self,nu): 
        # Function for adding noise to the signal
        s = self.s_nu(nu)
        self.sigma = 0.1*s
        return s + np.random.randn(len(nu))*self.sigma

    def gauss(self,var,d,p):
        """
        This is the amplitude sampler
        """
        mean_prior, sigma_prior = p
        if var=="A_d":
            T = self.s_d(self.nus)/getattr(self, var)
            d_ = d-self.s_cmb(self.nus)
        elif var=="A_cmb":
            T = self.s_cmb(self.nus)/getattr(self, var)
            d_ = d-self.s_d(self.nus)

        a1 = sum(d_*T/self.sigma**2+mean_prior/sigma_prior**2
                +T/self.sigma*np.random.normal()+np.random.normal()/sigma_prior)
        a2 = sum((T/self.sigma)**2+sigma_prior**-2)
        #a1 = sum(d_*T/self.sigma**2+T/self.sigma*np.random.normal())
        #a2 = sum((T/self.sigma)**2)
        setattr(self,var,a1/a2)

    def inverse(self, xs, com, d,p):
        """
        This is an inversion sampler for sampling spectral indices
        """
        mean_prior, sigma_prior = p
        P = np.zeros(1000)
        setattr(self,com,xs) # assign range of values
        for i, nu in enumerate(self.nus):
            s = self.s_nu(nu)
            P += (d[i]-s)**2/(2*(self.sigma[i])**2)
        
        P += (xs - mean_prior)**2 / (2 * sigma_prior**2)
        Px = np.exp(-P+np.min(P))
        dx = abs(xs[1]-xs[0])
        Fx = np.cumsum(Px*dx)
        eta = np.random.uniform(0,max(Fx))
        c1 = min(xs); c2 = max(xs)
        f = interp1d(Fx, xs, bounds_error=False, fill_value=(c1,c2))
        x = f(eta)
        if x < c1: x=c1
        elif x > c2: x=c2
        # Update value
        setattr(self,com,x)

    def output(self, i, out_every):
        if not i%out_every:
            print("---- Iteration", i, " -----")
            print(f"A_d  : {self.A_d:.2f}" )
            print(f"A_cmb: {self.A_cmb:.2f}")
            print(f"beta : {self.beta:.2f}")
            print(f"T    : {self.T:.2f}")

    def gibbs(self, d, params):
        for label, comp in params["comps"].items():
            for par, val in comp.items():
                # Set init value of parameter
                setattr(self, par, val["init"]) 

        n = params["ngibbs"]
        burnin = int(n/4) # When to start saving
        thetas = np.zeros((n-burnin,4)) # Saving data
        
        # Ranges for inverse sampling
        xbeta = np.linspace(1.0,3.0,1000)
        xT = np.linspace(10.,30.,1000)
        for i in tqdm(range(n), desc="Gibbs sampling"): # Gibbs samples
            # output every nth sample
            #self.output(i,out_every=200)

            """ Sampling A_dust """
            parameter = "A_d"
            if params["comps"]["dust"][parameter]["sample"]:
                self.gauss(parameter,d,params["comps"]["dust"][parameter]["prior"])
            else: 
                setattr(self,parameter, np.random.normal(*params["comps"]["dust"][parameter]["prior"]))

            """ Sampling A_cmb """
            parameter = "A_cmb"
            if params["comps"]["cmb"][parameter]["sample"]:
                self.gauss(parameter,d,params["comps"]["cmb"][parameter]["prior"])
            else: 
                setattr(self, parameter, np.random.normal(*params["comps"]["cmb"][parameter]["prior"]))

            """ Sampling Beta """
            parameter = "beta"
            if params["comps"]["dust"][parameter]["sample"]:
                self.inverse(xbeta, parameter, d, params["comps"]["dust"][parameter]["prior"])
            else: 
                setattr(self, parameter, np.random.normal(*params["comps"]["dust"][parameter]["prior"]))
            """ Sampling T """
            parameter = "T"
            if params["comps"]["dust"][parameter]["sample"]:
                self.inverse(xT, parameter,d,params["comps"]["dust"][parameter]["prior"])
            else: 
                setattr(self, parameter, np.random.normal(*params["comps"]["dust"][parameter]["prior"]))

            if i >= burnin: thetas[i-burnin,:] = (self.A_d, self.A_cmb, self.beta, self.T)
        return thetas
    


## Make simulated pixels on the sky for a chosen set of frequency bands

In [ ]:
# Simulated dataset of frequency bands
nus = np.array([30,44,70,100,143,217,353,545,857,3000]) #GHz
#nus = np.logspace(np.log10(10),np.log10(2500),50) #GHz
model = sky_model(nus)   # Initialize sky model
pixels = model.d_nu(nus) # Simulate pixels

## Sample new values initialized on wrong ones with the Gibbs sampler

In [ ]:
prior_scaling=1
params = {
    "ngibbs": 2000,
    "comps":{
        "cmb": {
            "A_cmb": {
                "sample": True,
                "init": 70,
                "prior": (60, 20*prior_scaling),
            }
        },
        "dust": {
            "A_d": {
                "sample": True,
                "init": 100,
                "prior": (110, 20*prior_scaling),
            },
            "beta": {
                "sample": True,
                "init": 1.8,
                "prior": (1.65, .3*prior_scaling),
            },
            "T": {
                "sample": True,
                "init": 20,
                "prior": (18, 3*prior_scaling),
            }
        }
    }
}
thetas = model.gibbs(pixels, params) # Start gibbs sampling with these init values

## Plot corner plot of sampled values


In [ ]:
print("Plotting")
fig = corner.corner(thetas,color='g', labels=[r"$A_d$",r"$A_{CMB}$", r"$\beta$", r"$T$"], show_titles=True, title_kwargs={"fontsize": 12})
plt.tight_layout()
plt.show()

### Plot Sampled sky model

In [ ]:

plt.loglog(nus,model.s_d(nus), color="C0", label="dust")
plt.loglog(nus,model.s_cmb(nus),color="C1", label="cmb")
model = sky_model(nus)   # Initialize sky model
plt.loglog(nus,model.s_d(nus),color="C0", linestyle=":", label="dust True")
plt.loglog(nus,model.s_cmb(nus),color="C1", linestyle=":", label="cmb True")
plt.scatter(nus,pixels, label="data with noise")
plt.legend(frameon=False)
nus_ = np.array([30,44,70,100,143,217,353,545,857,2000]) #GHz
plt.xticks(nus_,nus_)
plt.xlabel(r'$\nu$ [GHz]')
plt.ylabel('Signal amplitude')
plt.ylim(1e-1,1e3)

plt.grid()
plt.show()